# Two-Particle Self-Consistent approach (TPSC) tutorial

This tutorial is done in four steps

1. You will first learn how to manipulate multivariable Green's functions . You will also
   convince yourself that for nearest-neighbor non-interacting models, the Fermi surface has perfect nesting
   (TPSC-1)

2. You will compute the Lindhard function for the non-interacting susceptibility (TPSC-2)

3. You will then compute the RPA approximation to check the divergence at ($\pi,\pi)$ (TPSC-3)

4. Renormalized spin and charge vertices in TPSC are computed. (TPSC-4)
   The spin susceptibility is computed to show that it does not diverge at finite temperature.

5. A challenging exercice with the self-energy is left at the end if you have time. (TPSC-5)

## Lattice Green function

In this notebook, we will first manipulate the Green's function on a square lattice with nearest neighbour hopping $t$, 

\begin{equation}
G_0(\mathbf{k},i\omega_n)=\frac{1}{i\omega_n - \epsilon(\mathbf{k})}
\end{equation}

whoose dispersion is $\epsilon(\mathbf{k})=-2t(\cos{k_x}+\cos{k_y})$, where $\mathbf{k}$ is a vector in the Brillouin zone (in units where the lattice spacing is unity $a=1$) and $i\omega_n$ is a Matsubara frequency.

In [ ]:
%matplotlib inline
from pytriqs.plot.mpl_interface import plt
import numpy as np

In [ ]:
beta = 1./0.4
t = 1.0 
n_k = 128
n_w = 128

## A mesh on a Brillouin Zone

We first define a simple Bravais lattice (`BravaisLattice`) in 2 dimensions with basis vectors $\hat{e}_x = (1, 0, 0)$ and $\hat{e}_y=(0, 1, 0)$, and given the bravais lattice we construct the reciprocal (momentum) space Brillouin zone (`BrillouinZone`).

In [ ]:
from pytriqs.lattice import BravaisLattice, BrillouinZone
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)]) # Two unit vectors in R3
BZ = BrillouinZone(BL) 

The complex valued Green's function $G(\mathbf{k}, i\omega_n) $ is represented on the cartesian product mesh : $ (\mathbf{k} \times i\omega_n) \rightarrow {\mathcal{C}}$. 

We construct $G$ by first defining the two separate meshes in momentum space $\mathbf{k}$ (`MeshBrillouinZone`) and frequency space $i\omega_n$ (`MeshImFreq`) and then use the `MeshProduct` of these meshes as the mesh for $G(\mathbf{k}, i\omega_n)$.

In [ ]:
from pytriqs.gf import MeshBrillouinZone, MeshImFreq

kmesh = MeshBrillouinZone(BZ, n_k=n_k)
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

from pytriqs.gf import Gf, MeshProduct, Idx

g0 = Gf(mesh=MeshProduct(kmesh, wmesh), target_shape=[])

print g0

To fill the Green's function we construct a function for the dispersion $\epsilon(\mathbf{k})$ and set each element of $G$ by looping over the momentum and frequency meshes.

In [ ]:
def eps(k):
    return -2 * t* (np.cos(k[0]) + np.cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g0.mesh[0]:
    for w in g0.mesh[1]:
        g0[k, w] = 1/(w - eps(k))

## (Optional) More advanced

- We could show how to acces g0.data to vectorize its initialization
- We could show a first example of Magic here

## Save Green function for later use

In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("tpsc.h5") as R:
    R['g0_kw'] = g0

## For nearest-neighbor model, the Fermi surface is nested

### Plot of the momentum distribution curve at the Fermi level

The Fermi surface is nested. 

    What do we mean by that?
    What is the nesting vector?

In [ ]:
k = np.linspace(-np.pi, np.pi, n_k+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

spectral = lambda kx, ky: -g0([kx,ky,0],0).imag
fs = lambda kx, ky: (1/g0([kx,ky,0],0)).real

plt.figure(figsize=(7,7))
plt.pcolor(kx, ky, np.vectorize(spectral)(kx,ky))
plt.colorbar()
plt.contour(kx, ky, np.vectorize(fs)(kx,ky), levels=[0], colors='white')
plt.axes().set_aspect('equal')

# Cosmetics
plt.xticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])    
plt.yticks([-np.pi, 0, np.pi],[r"$-\pi$", r"0", r"$\pi$"])
plt.xlabel(r"$k_x$")
plt.ylabel(r"$k_y$")
plt.title("Momentum distribution curve (MDC) at the Fermi level")

## Density (TODO)

It would be nice to show the density along a diagonal in $k$ to see the signature of the Fermi surface.

TODO: use ``closest_mesh_point``

In [ ]:
d = lambda k: g0[Idx(k,k,0),:].density().real
kr = range(n_k/2)

plt.plot(kr, np.vectorize(d)(kr), '-o')

## Simple tests (will be removed)

In [ ]:
k = np.linspace(0, 2*np.pi, n_k+1, endpoint=True)
check = lambda kx, ky: g0([kx,ky,0],0).imag

# Should be 0
print g0.data.reshape(n_k, n_k, 2*n_w)[:,0,n_w].imag - np.vectorize(check)(k,0)[0:n_k]

mf = lambda n : (2*n+1)*np.pi/beta*1j
for k in g0.mesh[0]:
    assert abs(g0(k.value, 0) - 1/(mf(0) - eps(k)))< 1.e-10